In [81]:
import numpy as np
import pandas as pd

!pip install git+https://github.com/microsoft/recommenders.git
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)


  Cloning https://github.com/microsoft/recommenders.git to /tmp/pip-req-build-5k6og_0n
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/recommenders.git /tmp/pip-req-build-5k6og_0n
  Resolved https://github.com/microsoft/recommenders.git to commit c2ea583d27bb1a4d58a09a1621d5ce95672ef1dc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [82]:
import sys
print(sys.version)


3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [100]:
!gdown https://drive.google.com/uc?id=1vbteMX-kHyYAAS7JIrrog_aZvvOY77Q0

df = pd.read_csv('final_dataset.csv')
df.head()





Downloading...
From (original): https://drive.google.com/uc?id=1vbteMX-kHyYAAS7JIrrog_aZvvOY77Q0
From (redirected): https://drive.google.com/uc?id=1vbteMX-kHyYAAS7JIrrog_aZvvOY77Q0&confirm=t&uuid=91848008-d8c7-4311-89bd-ce66c4b8508e
To: /content/final_dataset.csv
100% 456M/456M [00:04<00:00, 99.5MB/s]


,Movie,Reviewer,Review,Score,Sentiment_scores,overview,cast_list,crew_list,director_list,first_three_casts,unique_genres,Date,recommend
0,SENSATION,Dennis Schwartz,"A sci-fi film that made no sense, as it tells ...",0.686869,0.276348,Man-boy farmer Donal discovers that his father...,"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe'...",['Tom Hall'],['Tom Hall'],"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe']","['Drama', 'Romance', 'Comedy', 'Foreign']",2022-09-01,0
1,SENSATION,Carey-Ann Pawsey,A thriller that ultimately makes little to no ...,0.393939,0.334176,Man-boy farmer Donal discovers that his father...,"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe'...",['Tom Hall'],['Tom Hall'],"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe']","['Drama', 'Romance', 'Comedy', 'Foreign']",2022-06-01,0
2,SENSATION,Rob Rector,"There's still much to admire with Sensation, d...",0.595960,0.739928,Man-boy farmer Donal discovers that his father...,"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe'...",['Tom Hall'],['Tom Hall'],"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe']","['Drama', 'Romance', 'Comedy', 'Foreign']",2021-05-19,0
3,SENSATION,Rich Cline,There's no thematic depth in the story or char...,0.494949,0.349340,Man-boy farmer Donal discovers that his father...,"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe'...",['Tom Hall'],['Tom Hall'],"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe']","['Drama', 'Romance', 'Comedy', 'Foreign']",2021-04-22,0
4,SENSATION,Allen Adams,Sensation wants to be a deep movie...[but] tho...,0.393939,0.240863,Man-boy farmer Donal discovers that his father...,"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe'...",['Tom Hall'],['Tom Hall'],"['Luanne Gordon', 'Kelly Campbell', 'Owen Roe']","['Drama', 'Romance', 'Comedy', 'Foreign']",2021-04-21,0


In [84]:
!pip install surprise
import surprise
from surprise import Dataset, Reader

In [85]:
# Split the data train 85% - test 15%
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42, shuffle=True)

print(len(train_df))

train_df.head()


239590


,Movie,Reviewer,Review,Score,Sentiment_scores,overview,cast_list,crew_list,director_list,first_three_casts,unique_genres,Date,recommend
177863,GOON,Robert Abele,Scott isn't bad: He sells the comedy of his ch...,0.494949,0.828236,"Doug Glatt, a slacker who discovers he has a t...","['Seann William Scott', 'Jay Baruchel', 'Aliso...","['Jay Baruchel', 'Lucie Robitaille', 'Don Carm...",['Michael Dowse'],"['Seann William Scott', 'Jay Baruchel', 'Aliso...",['Comedy'],2012-03-29,0
85979,TERMINATOR GENISYS,Michael O'Sullivan,Genisys goes back to what made the franchise w...,0.621212,0.498964,"The year is 2029. John Connor, leader of the r...","['Arnold Schwarzenegger', 'Jason Clarke', 'Emi...","['Ronna Kress', 'Gale Anne Hurd', 'James Camer...",['Alan Taylor'],"['Arnold Schwarzenegger', 'Jason Clarke', 'Emi...","['Science Fiction', 'Thriller', 'Adventure', '...",2015-06-30,0
126165,SABOTAGE,Adam Woodward,While Sabotage boasts the best of post-hiatus ...,0.393939,0.372593,"In ""Sabotage"", Arnold Schwarzenegger leads an ...","['Arnold Schwarzenegger', 'Sam Worthington', '...","['Dody Dorn', 'Mary Vernieu', 'Marc Fishman', ...",['David Ayer'],"['Arnold Schwarzenegger', 'Sam Worthington', '...","['Drama', 'Thriller', 'Crime', 'Action']",2014-08-05,0
147060,GETAWAY,Mary Houlihan,After a while it all becomes mind-numbing. How...,0.494949,0.498964,Former race car driver Brent Magna (Hawke) is ...,"['Ethan Hawke', 'Selena Gomez', 'Jon Voight', ...","['Justin Caine Burnett', 'Moshe Diamant', 'Cou...",['Courtney Solomon'],"['Ethan Hawke', 'Selena Gomez', 'Jon Voight']","['Crime', 'Action']",2013-08-29,0
276383,PATHOLOGY,John Campea,"An intense, sexy medical thriller that gives u...",0.747475,0.721579,Medical student Ted Grey (Milo Ventimiglia) gr...,"['Milo Ventimiglia', 'Alyssa Milano', 'Michael...","['Tom Rosenberg', 'Gary Lucchesi', 'Nancy Nayo...",['Marc Schölermann'],"['Milo Ventimiglia', 'Alyssa Milano', 'Michael...","['Thriller', 'Crime', 'Horror']",2008-04-15,0


In [86]:
rating_scale = (train_df['Score'].min(), train_df['Score'].max())
reader = Reader(rating_scale=rating_scale)

train_set = Dataset.load_from_df(train_df[['Reviewer', 'Movie', 'Score']], reader).build_full_trainset()
test_set = Dataset.load_from_df(test_df[['Reviewer', 'Movie', 'Score']], reader).build_full_trainset()



### **SVD Model**

In [87]:
from surprise import SVD
svd = SVD(random_state=42, n_factors=200, n_epochs=10, verbose=True)
svd.fit(train_set)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [88]:
# Predict using the train set
trainset = [tuple(x) for x in train_df[['Reviewer', 'Movie', 'Score']].to_numpy()]
predictions_svd_train = svd.test(trainset)
predictions_svd_train

[Prediction(uid='Robert Abele', iid='GOON', r_ui=0.494949494949495, est=0.5294637762317549, details={'was_impossible': False}),
 Prediction(uid="Michael O'Sullivan", iid='TERMINATOR GENISYS', r_ui=0.6212121212121212, est=0.45464555942329477, details={'was_impossible': False}),
 Prediction(uid='Adam Woodward', iid='SABOTAGE', r_ui=0.3939393939393939, est=0.45494809426981664, details={'was_impossible': False}),
 Prediction(uid='Mary Houlihan', iid='GETAWAY', r_ui=0.494949494949495, est=0.29693712388661114, details={'was_impossible': False}),
 Prediction(uid='John Campea', iid='PATHOLOGY', r_ui=0.7474747474747475, est=0.6050229188986451, details={'was_impossible': False}),
 Prediction(uid='Kofi Outlaw', iid='SABOTAGE', r_ui=0.2929292929292929, est=0.538637170041083, details={'was_impossible': False}),
 Prediction(uid='Angie Errigo', iid='THE LONE RANGER', r_ui=0.797979797979798, est=0.4979675070041117, details={'was_impossible': False}),
 Prediction(uid='Peter Canavese', iid='HARRY POTTER

In [89]:
# Predict using the test set
testset = [tuple(x) for x in test_df[['Reviewer', 'Movie', 'Score']].to_numpy()]
predictions_svd_test = svd.test(testset)
predictions_svd_test


[Prediction(uid='Bernard Boo', iid='BLUE CAPRICE', r_ui=0.6666666666666666, est=0.7301141947268674, details={'was_impossible': False}),
 Prediction(uid='John Hanlon', iid='WAR FOR THE PLANET OF THE APES', r_ui=0.8737373737373737, est=0.7777515955617778, details={'was_impossible': False}),
 Prediction(uid='Peter Bradshaw', iid='CINDERELLA', r_ui=0.5959595959595959, est=0.41092617372309814, details={'was_impossible': False}),
 Prediction(uid='Jeff Vice', iid='THE JONESES', r_ui=0.7474747474747475, est=0.4245411431127932, details={'was_impossible': False}),
 Prediction(uid='Lawrence Toppman', iid='BOLT', r_ui=0.8737373737373737, est=0.8504476296514543, details={'was_impossible': False}),
 Prediction(uid='Linda Cook', iid='AMELIA', r_ui=0.7474747474747475, est=0.5253870549197712, details={'was_impossible': False}),
 Prediction(uid='Brian Orndorf', iid='THE READER', r_ui=0.8383838383838383, est=0.7516979384665786, details={'was_impossible': False}),
 Prediction(uid='Dominick Suzanne-Mayer',

In [90]:
from surprise import accuracy
from sklearn.metrics import f1_score, precision_score, recall_score

# Evaluation of SVD Model Test Set
print('SVD Train Set Evaluation Metrics')

def rating_grouping(rating, threshold=0.797980):
    if (rating >= threshold):
        return 1
    else:
        return 0

# Extract true and predicted ratings from the predictions
real_user_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.r_ui)) for pred in predictions_svd_train], columns=['Reviewer', 'Movie', 'Actual_Rating'])
predicted_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.est)) for pred in predictions_svd_train], columns=['Reviewer', 'Movie', 'Predicted_Rating'])

# Precision
precision = precision_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Precision: {precision}')

# Recall
recall = recall_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Recall: {recall}')


#F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f'F1 Score: {f1}')


SVD Train Set Evaluation Metrics
Precision: 0.5640454346876365
Recall: 0.729880963490086
F1 Score: 0.6363360789327538


In [91]:
from surprise import accuracy
from sklearn.metrics import f1_score, precision_score, recall_score

# Evaluation of SVD Model Test Set
print('SVD Test Set Evaluation Metrics')

def rating_grouping(rating, threshold=0.797980):
    if (rating >= threshold):
        return 1
    else:
        return 0

# Extract true and predicted ratings from the predictions
real_user_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.r_ui)) for pred in predictions_svd_test], columns=['Reviewer', 'Movie', 'Actual_Rating'])
predicted_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.est)) for pred in predictions_svd_test], columns=['Reviewer', 'Movie', 'Predicted_Rating'])

# Precision
precision = precision_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Precision: {precision}')

# Recall
recall = recall_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Recall: {recall}')


#F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f'F1 Score: {f1}')



SVD Test Set Evaluation Metrics
Precision: 0.48537451189208375
Recall: 0.8313248540693758
F1 Score: 0.6129020879007422


In [92]:
import pandas as pd

def get_top_n(predictions, n=10):

    # Map predictions to each user.
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the N highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n_recommendations = get_top_n(predictions_svd_test, n=10)

def show_recommendations(user_id):
    print(f"Top recommendations for Reviewer {user_id}:")
    top_recs = top_n_recommendations[user_id]
    for movie, rating in top_recs:
        print(f"Movie Name: {movie}, Predicted Rating: {rating:.4f}")

# Demonstration to show top 10 recommendations for a user -> Using 'Dennis Schwartz' as an example
show_recommendations('Dennis Schwartz')


Top recommendations for Reviewer Dennis Schwartz:
Movie Name: 12 YEARS A SLAVE, Predicted Rating: 0.9853
Movie Name: MANCHESTER BY THE SEA, Predicted Rating: 0.9783
Movie Name: TIMBUKTU, Predicted Rating: 0.9782
Movie Name: I AM NOT YOUR NEGRO, Predicted Rating: 0.9748
Movie Name: WHIPLASH, Predicted Rating: 0.9589
Movie Name: UP, Predicted Rating: 0.9543
Movie Name: STORIES WE TELL, Predicted Rating: 0.9401
Movie Name: THE WRESTLER, Predicted Rating: 0.9344
Movie Name: ARGO, Predicted Rating: 0.9304
Movie Name: CAT PEOPLE, Predicted Rating: 0.9292


### **SVD++ Model**


In [93]:
# To use the same train_set and test_set from the SVD model.
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42, shuffle=True)

rating_scale = (train_df['Score'].min(), train_df['Score'].max())
reader = Reader(rating_scale=rating_scale)

train_set = Dataset.load_from_df(train_df[['Reviewer', 'Movie', 'Score']], reader).build_full_trainset()
test_set = Dataset.load_from_df(test_df[['Reviewer', 'Movie', 'Score']], reader).build_full_trainset()

In [94]:
from surprise import SVDpp
svdpp = SVDpp(random_state=42, n_factors=200, n_epochs=10, verbose=True)
svdpp.fit(train_set)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9


In [95]:
# Predict using the train set
trainset = [tuple(x) for x in train_df[['Reviewer', 'Movie', 'Score']].to_numpy()]
predictions_svdpp_train = svdpp.test(trainset)
predictions_svdpp_train

[Prediction(uid='Robert Abele', iid='GOON', r_ui=0.494949494949495, est=0.5384170251376176, details={'was_impossible': False}),
 Prediction(uid="Michael O'Sullivan", iid='TERMINATOR GENISYS', r_ui=0.6212121212121212, est=0.4435604657648991, details={'was_impossible': False}),
 Prediction(uid='Adam Woodward', iid='SABOTAGE', r_ui=0.3939393939393939, est=0.4424764640707677, details={'was_impossible': False}),
 Prediction(uid='Mary Houlihan', iid='GETAWAY', r_ui=0.494949494949495, est=0.32321173689311067, details={'was_impossible': False}),
 Prediction(uid='John Campea', iid='PATHOLOGY', r_ui=0.7474747474747475, est=0.6156753442049836, details={'was_impossible': False}),
 Prediction(uid='Kofi Outlaw', iid='SABOTAGE', r_ui=0.2929292929292929, est=0.5219790407056404, details={'was_impossible': False}),
 Prediction(uid='Angie Errigo', iid='THE LONE RANGER', r_ui=0.797979797979798, est=0.5080849455408528, details={'was_impossible': False}),
 Prediction(uid='Peter Canavese', iid='HARRY POTTER 

In [96]:
# Predict using the test set
testset = [tuple(x) for x in test_df[['Reviewer', 'Movie', 'Score']].to_numpy()]
predictions_svdpp_test = svdpp.test(testset)
predictions_svdpp_test


[Prediction(uid='Bernard Boo', iid='BLUE CAPRICE', r_ui=0.6666666666666666, est=0.7357045781176437, details={'was_impossible': False}),
 Prediction(uid='John Hanlon', iid='WAR FOR THE PLANET OF THE APES', r_ui=0.8737373737373737, est=0.7965276998831741, details={'was_impossible': False}),
 Prediction(uid='Peter Bradshaw', iid='CINDERELLA', r_ui=0.5959595959595959, est=0.41800171699734434, details={'was_impossible': False}),
 Prediction(uid='Jeff Vice', iid='THE JONESES', r_ui=0.7474747474747475, est=0.45712254154456333, details={'was_impossible': False}),
 Prediction(uid='Lawrence Toppman', iid='BOLT', r_ui=0.8737373737373737, est=0.830961511949508, details={'was_impossible': False}),
 Prediction(uid='Linda Cook', iid='AMELIA', r_ui=0.7474747474747475, est=0.5357245721182095, details={'was_impossible': False}),
 Prediction(uid='Brian Orndorf', iid='THE READER', r_ui=0.8383838383838383, est=0.7540706430291974, details={'was_impossible': False}),
 Prediction(uid='Dominick Suzanne-Mayer',

In [97]:
# Evaluation of SVD++ Model Train Set
print('SVD++ Train Set Evaluation Metrics')


def rating_grouping(rating, threshold=0.797980):
    if (rating >= threshold):
        return 1
    else:
        return 0

# Extract true and predicted ratings from the predictions
real_user_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.r_ui)) for pred in predictions_svdpp_train], columns=['Reviewer', 'Movie', 'Actual_Rating'])
predicted_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.est)) for pred in predictions_svdpp_train], columns=['Reviewer', 'Movie', 'Predicted_Rating'])

# Precision
precision = precision_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Precision: {precision}')

# Recall
recall = recall_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Recall: {recall}')


#F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f'F1 Score: {f1}')

SVD++ Train Set Evaluation Metrics
Precision: 0.5640235910878113
Recall: 0.7298906260089439
F1 Score: 0.6363258499571943


In [98]:
# Evaluation of SVD++ Model

print('SVD++ Test Set Evaluation Metrics')

def rating_grouping(rating, threshold=0.797980):
    if (rating >= threshold):
        return 1
    else:
        return 0

# Extract true and predicted ratings from the predictions
real_user_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.r_ui)) for pred in predictions_svdpp_test], columns=['Reviewer', 'Movie', 'Actual_Rating'])
predicted_ratings = pd.DataFrame([(pred.uid, pred.iid, rating_grouping(pred.est)) for pred in predictions_svdpp_test], columns=['Reviewer', 'Movie', 'Predicted_Rating'])

# Precision
precision = precision_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Precision: {precision}')

# Recall
recall = recall_at_k(rating_pred = predicted_ratings, rating_true = real_user_ratings, k = 10, col_user='Reviewer', col_item='Movie', col_rating='Actual_Rating', col_prediction = 'Predicted_Rating')
print(f'Recall: {recall}')


#F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f'F1 Score: {f1}')


SVD++ Test Set Evaluation Metrics
Precision: 0.4854100106496273
Recall: 0.8313273022595511
F1 Score: 0.6129310541850929


In [99]:
import pandas as pd

def get_top_n(predictions, n=10):

    # Map predictions to each user.
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the N highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n_recommendations = get_top_n(predictions_svd, n=10)

def show_recommendations(user_id):
    print(f"Top recommendations for Reviewer {user_id}:")
    top_recs = top_n_recommendations[user_id]
    for movie, rating in top_recs:
        print(f"Movie Name: {movie}, Predicted Rating: {rating:.4f}")

# Demonstration to show top 10 recommendations for a user -> Using 'Dennis Schwartz' as an example
show_recommendations('Dennis Schwartz')

Top recommendations for Reviewer Dennis Schwartz:
Movie Name: 12 YEARS A SLAVE, Predicted Rating: 0.9853
Movie Name: MANCHESTER BY THE SEA, Predicted Rating: 0.9783
Movie Name: TIMBUKTU, Predicted Rating: 0.9782
Movie Name: I AM NOT YOUR NEGRO, Predicted Rating: 0.9748
Movie Name: WHIPLASH, Predicted Rating: 0.9589
Movie Name: UP, Predicted Rating: 0.9543
Movie Name: STORIES WE TELL, Predicted Rating: 0.9401
Movie Name: THE WRESTLER, Predicted Rating: 0.9344
Movie Name: ARGO, Predicted Rating: 0.9304
Movie Name: CAT PEOPLE, Predicted Rating: 0.9292


### **Conclusion**

The SVD++ model shows a *negligible* improvement in all the evaluation metrics over the SVD model. This can be attributed to the SVD++ model's ability to leverage implicit feedback, which may provide it with a more comprehensive understanding of user preferences. However, the difference in performance is not substantial, indicating that both models are performing similarly in your specific context.

Additionally, we think that the SVD model would even be more time-cost efficient as the evaluation scores between the SVD and SVD++ is not significantly different, given that SVD++ is a more complexed variant and takes a longer time to run the epochs.